<a href="https://colab.research.google.com/github/Nickchiu120026/2025_machine_learning/blob/main/Homework_6_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import xml.etree.ElementTree as ET
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset, random_split
import numpy as np
import matplotlib.pyplot as plt

# ---------- 1. 讀 XML ----------
xml_file = "O-A0038-003.xml"
tree = ET.parse(xml_file)
root = tree.getroot()
ns = {'cwa': 'urn:cwa:gov:tw:cwacommon:0.1'}

lon0 = float(root.find('.//cwa:BottomLeftLongitude', ns).text)
lat0 = float(root.find('.//cwa:BottomLeftLatitude', ns).text)
content_text = root.find('.//cwa:Content', ns).text.strip()
values = [float(v) for v in content_text.replace('\n', ',').split(',') if v.strip()]

cols, rows = 67, 120
grid = np.array(values).reshape(rows, cols)
label_grid = np.where(grid == -999.0, 0, 1)
value_grid = np.where(grid == -999.0, np.nan, grid)

lon = np.linspace(lon0, lon0 + (cols - 1) * 0.03, cols)
lat = np.linspace(lat0, lat0 + (rows - 1) * 0.03, rows)

# ---------- 2. 可視化 ----------
fig, axes = plt.subplots(1, 2, figsize=(14, 6))
im1 = axes[0].imshow(label_grid, extent=[lon.min(), lon.max(), lat.min(), lat.max()],
               origin='lower', cmap='gray', vmin=0, vmax=1)
axes[0].set_title("Valid Label (1=valid, 0=invalid)")
axes[0].set_xlabel("Longitude (°E)")
axes[0].set_ylabel("Latitude (°N)")

# 自定義 colorbar 的 ticks 與 label
axes[0].text(1.02, 1.0, "White = 1\nBlack = 0", transform=axes[0].transAxes, fontsize=12, va='top', ha='left')

im2 = axes[1].imshow(value_grid, extent=[lon.min(), lon.max(), lat.min(), lat.max()],
                     origin='lower', cmap='jet')
axes[1].set_title("Temperature Value (°C)")
axes[1].set_xlabel("Longitude (°E)")
axes[1].set_ylabel("Latitude (°N)")
fig.colorbar(im2, ax=axes[1], label="Temperature (°C)")
plt.tight_layout()
plt.savefig("Original.png", dpi=300)
plt.show()

# ---------- 3. 整理資料 ----------
lon_mesh, lat_mesh = np.meshgrid(lon, lat)
lon_flat = lon_mesh.flatten()
lat_flat = lat_mesh.flatten()
value_flat = grid.flatten()
label_flat = (value_flat != -999.0).astype(float)

lon_min, lon_max = lon.min(), lon.max()
lat_min, lat_max = lat.min(), lat.max()
def norm_coords(lon_vals, lat_vals):
    lon_n = (lon_vals - lon_min) / (lon_max - lon_min)
    lat_n = (lat_vals - lat_min) / (lat_max - lat_min)
    return lon_n, lat_n

lon_norm, lat_norm = norm_coords(lon_flat, lat_flat)

tensor_cls = torch.tensor(np.column_stack([lon_norm, lat_norm, label_flat]), dtype=torch.float32)
tensor_reg = torch.tensor(np.column_stack([
    lon_norm[label_flat==1], lat_norm[label_flat==1], value_flat[label_flat==1]
]), dtype=torch.float32)

# ---------- 4. Dataset ----------
def make_dataset(tensor, batch_size=64, split=0.8):
    X = tensor[:, :2]
    y = tensor[:, -1].unsqueeze(1)
    dataset = TensorDataset(X, y)
    n_train = int(len(dataset) * split)
    n_val = len(dataset) - n_train
    train_ds, val_ds = random_split(dataset, [n_train, n_val])
    return (DataLoader(train_ds, batch_size=batch_size, shuffle=True),
            DataLoader(val_ds, batch_size=batch_size))

train_cls, val_cls = make_dataset(tensor_cls)
train_reg, val_reg = make_dataset(tensor_reg)

# ---------- 5. 模型 ----------
class Classifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(2, 128), nn.ReLU(),
            nn.Linear(128, 64), nn.ReLU(),
            nn.Linear(64, 32), nn.ReLU(),
            nn.Linear(32, 8), nn.ReLU(),
            nn.Linear(8, 1)  # 不加 Sigmoid
        )
    def forward(self, x):
        return self.net(x)  # raw logits

class Regressor(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(2, 64), nn.Tanh(),
            nn.Linear(64, 64), nn.Tanh(),
            nn.Linear(64, 32), nn.Tanh(),
            nn.Linear(32, 8), nn.Tanh(),
            nn.Linear(8, 1)
        )
    def forward(self, x):
        return self.net(x)

class HxModel(nn.Module):
    def __init__(self, classifier, regressor, invalid_value=-999.0, threshold=0.5):
        super().__init__()
        self.classifier = classifier
        self.regressor = regressor
        self.invalid_value = invalid_value
        self.threshold = threshold
    def forward(self, x):
        c_logit = self.classifier(x)
        c_pred = torch.sigmoid(c_logit)
        r_pred = self.regressor(x)
        return torch.where(c_pred > self.threshold, r_pred, torch.tensor(self.invalid_value, device=x.device))

# ---------- 6. 訓練 ----------
classifier = Classifier()
opt_c = torch.optim.Adam(classifier.parameters(), lr=1e-3)
loss_fn_c = nn.BCEWithLogitsLoss()

train_loss_c, val_loss_c = [], []
for epoch in range(1, 1001):
    classifier.train()
    total = 0
    for X, y in train_cls:
        opt_c.zero_grad()
        pred = classifier(X)
        loss = loss_fn_c(pred, y)
        loss.backward()
        opt_c.step()
        total += loss.item()
    train_loss_c.append(total / len(train_cls))

    classifier.eval()
    with torch.no_grad():
        val_total = 0
        for Xv, yv in val_cls:
            pv = classifier(Xv)
            val_total += loss_fn_c(pv, yv).item()
        val_loss_c.append(val_total / len(val_cls))

    if epoch % 50 == 0:
        print(f"[Classifier] Epoch {epoch:03d} | Train={train_loss_c[-1]:.4f} | Val={val_loss_c[-1]:.4f}")

# ---------- 7. Regressor ----------
regressor = Regressor()
opt_r = torch.optim.Adam(regressor.parameters(), lr=1e-3)
loss_fn_r = nn.MSELoss()
train_loss_r, val_loss_r = [], []

for epoch in range(1, 3001):
    regressor.train()
    total = 0
    for X, y in train_reg:
        opt_r.zero_grad()
        pred = regressor(X)
        loss = loss_fn_r(pred, y)
        loss.backward()
        opt_r.step()
        total += loss.item()
    train_loss_r.append(total / len(train_reg))

    regressor.eval()
    with torch.no_grad():
        val_total = 0
        for Xv, yv in val_reg:
            pv = regressor(Xv)
            val_total += loss_fn_r(pv, yv).item()
        val_loss_r.append(val_total / len(val_reg))

    if epoch % 50 == 0:
        print(f"[Regressor] Epoch {epoch:03d} | Train={train_loss_r[-1]:.4f} | Val={val_loss_r[-1]:.4f}")

# ---------- 8. 預測與可視化 ----------
piecewise_model = HxModel(classifier, regressor)

test_points = torch.tensor(np.column_stack([lon_norm, lat_norm]), dtype=torch.float32)
piecewise_model.eval()
with torch.no_grad():
    h_pred_grid = piecewise_model(test_points).reshape(rows, cols).numpy()

fig, axes = plt.subplots(1, 2, figsize=(8, 5), constrained_layout=True)

im0 = axes[0].imshow(grid, extent=[lon.min(), lon.max(), lat.min(), lat.max()],
                     origin='lower', cmap='jet', vmin=0, vmax=30)
axes[0].set_title("True Data")

im2 = axes[1].imshow(h_pred_grid, extent=[lon.min(), lon.max(), lat.min(), lat.max()],
                     origin='lower', cmap='jet', vmin=0, vmax=30)
axes[1].set_title("h(x) Result")

plt.savefig("Result.png", dpi=300)
plt.show()


In [ ]:
import numpy as np
from matplotlib.patches import Ellipse
import matplotlib.pyplot as plt
import math

class GDA:
    def __init__(self):
        self.phi = None      # P(y=1)
        self.mu0 = None      # mean of class 0
        self.mu1 = None      # mean of class 1
        self.S = None        # shared covariance matrix
        self.S1 = None       # for ellipse only

    def fit(self, X, y):
        X0 = X[y == 0]
        X1 = X[y == 1]
        self.mu0 = np.mean(X0, axis=0)
        self.mu1 = np.mean(X1, axis=0)
        self.phi = len(X1) / len(X)
        S0 = np.cov(X0.T, bias=True)
        S1 = np.cov(X1.T, bias=True)
        self.S = (len(X0) * S0 + len(X1) * S1) / len(X)  # shared covariance
        self.S1 = S1

        # 預先計算 inverse
        self.S_inv = np.linalg.inv(self.S)

    def predict_proba(self, X):
        """Compute P(y=1|x)"""
        def g(x):
            term1 = -0.5 * (x - self.mu0) @ self.S_inv @ (x - self.mu0).T
            term2 = -0.5 * (x - self.mu1) @ self.S_inv @ (x - self.mu1).T
            return np.exp(term2) * self.phi / (np.exp(term2) * self.phi + np.exp(term1) * (1 - self.phi))

        return np.array([g(x) for x in X])

def plot_gda_boundary_and_ellipse(data_tensor, gda, lon_vals, lat_vals):
    data = data_tensor.numpy()
    X = data[:, :2]
    y = data[:, 2]

    lon_min, lon_max = lon_vals.min(), lon_vals.max()
    lat_min, lat_max = lat_vals.min(), lat_vals.max()

    LON, LAT = np.meshgrid(lon_vals, lat_vals)
    norm_lon = (LON - lon_min) / (lon_max - lon_min)
    norm_lat = (LAT - lat_min) / (lat_max - lat_min)
    gridX = np.column_stack([norm_lon.flatten(), norm_lat.flatten()])

    post = gda.predict_proba(gridX).reshape(LON.shape)

    fig, ax = plt.subplots(figsize=(6, 9))

    ax.contourf(LON, LAT, post, levels=[0, 0.5, 1], colors=['lightcoral', 'midnightblue'], alpha=0.3)

    mask0 = (y == 0)
    mask1 = (y == 1)
    ax.scatter(X[mask0, 0] * (lon_max - lon_min) + lon_min,
               X[mask0, 1] * (lat_max - lat_min) + lat_min,
               color='red', s=5, label='Class 0: No Data')
    ax.scatter(X[mask1, 0] * (lon_max - lon_min) + lon_min,
               X[mask1, 1] * (lat_max - lat_min) + lat_min,
               color='navy', s=5, label='Class 1: Has Data')

    mu = gda.mu1
    S = gda.S1
    vals, vecs = np.linalg.eigh(S)
    order = np.argsort(vals)[::-1]
    vals = vals[order]
    vecs = vecs[:, order]
    angle = np.degrees(np.arctan2(vecs[1, 0], vecs[0, 0]))
    k = math.sqrt(5.991)  # chi^2_2 at 95%
    width = 2 * k * np.sqrt(vals[0])
    height = 2 * k * np.sqrt(vals[1])

    center_lon = mu[0] * (lon_max - lon_min) + lon_min
    center_lat = mu[1] * (lat_max - lat_min) + lat_min

    scale_mat = np.diag([lon_max - lon_min, lat_max - lat_min])
    scaled_cov = scale_mat @ S @ scale_mat.T
    vals, vecs = np.linalg.eigh(scaled_cov)
    order = np.argsort(vals)[::-1]
    vals = vals[order]
    vecs = vecs[:, order]

    angle = np.degrees(np.arctan2(vecs[1, 0], vecs[0, 0]))
    k = math.sqrt(5.991)  # chi2_2,0.95
    width = 2 * k * np.sqrt(vals[0])
    height = 2 * k * np.sqrt(vals[1])

    ellipse = Ellipse((center_lon, center_lat),
                      width=width,
                      height=height,
                      angle=angle,
                      edgecolor='yellow', facecolor='none', lw=2)
    ax.add_patch(ellipse)
    ax.set_title("Class-1 Gaussian 95% Contour (original lon/lat)")
    ax.set_xlabel("Longitude (°E)")
    ax.set_ylabel("Latitude (°N)")
    ax.legend(loc='upper left')
    plt.tight_layout()
    plt.savefig("GDA_Contour.png", dpi=300)
    plt.show()


X_cls = tensor_cls[:, :2].numpy()
y_cls = tensor_cls[:, 2].numpy()

gda = GDA()
gda.fit(X_cls, y_cls)

plot_gda_boundary_and_ellipse(tensor_cls, gda, lon, lat)